In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
import re
import pickle

In [2]:
with open ('all_therapists1000.txt', 'rb') as fp:
    all_therapists = pickle.load(fp)

In [3]:
final_df = pd.DataFrame(columns=['statement', 'name', 'verified', 'degree', 'addr', 'city', 'state', 'zip', 'phone', 'cost_per_ses','sliding_scale','pay_by','accepted_insurance', 'specs', 'issues', 'mental_health', 'sexuality', 'client_focus_age', 'client_focus_communities', 'treatment_approach_types_of_ther', 'treatment_approach_modality', 'cert', 'cert_date', 'ethnicity', 'language', 'faith', 'tele', 'no_tele'])

In [4]:
def scraper(urls, path = r'C:/Users/Tinatin/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe'):
    global final_df
    #iterate over urls
    count = 0
    for url in urls:
        count += 1
        print(count)
        print(url)
        row_results = {}
        driver = Chrome(executable_path= path)
        try:
            driver.get(url) #open the url in selenium
             #wait for the webpage to load
            time.sleep(5)
        except:
            print ('bad url!')
        soup = BeautifulSoup(driver.page_source,'html.parser') 
       
        #row_results['full_address'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = "address-data").text.split('\n') if item) if item])
        try:   
            row_results['state'] = soup.find(itemprop = "addressRegion").text
        except:
            print('state field missing')
            row_results['state'] = 'missing'
        try:
            row_results['addr'] = [item for item in (item.strip() for item in soup.find(itemprop = 'streetAddress').text.split('\n') if item) if item][0]
        except:
            print('street address field missing')
            row_results['addr'] = 'missing'
        try:
            row_results['city'] = soup.find(itemprop = 'addressLocality').text
        except:
            row_results['city'] = 'missing'
            print('city field missing')
        try:
            row_results['zip'] = soup.find(itemprop = 'postalcode').text
        except:
            print('zip field missing')
            row_results['zip'] = 'missing'
        #row_results['header_info'] = ', '.join([item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item])
        try:
            row_results['name'] = [item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][0]
        except:
            row_results['name'] = 'missing'
            print('name missing')
        try:
            row_results['degree'] = ', '.join([item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][1:-1])
        except:
            row_results['degree'] = 'missing'
            print('degree missing')
        try:
            row_results['verified'] = [item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][-1]
        except:
            row_results['verified'] = 'missing'
            print('verified missing')
        try:
            row_results['phone'] = soup.find(class_ = "phone-number").text
        except:
            row_results['phone'] = 'missing'
            print('phone missing')
        try:
            row_results['statement'] = ''.join([item for item in soup.find(class_ = 'section profile-personalstatement').text.split('\n') if item])
        except:
            print('statement missing')
            row_results['statement'] = 'missing'
        try:
            finances = [item for item in (item.strip() for item in soup.find(id = 'tabs-finances-office').text.split('\n') if item) if item]
            for i in range(len(finances)):
                if 'Verify your health' in finances[i]:
                    finances.remove(finances[-1])
        except:
            finances = []
        #print(finances)
        
        if any("Cost per Session" in s for s in finances):
            for i in range(len(finances)):
                if 'Cost per Session: ' in finances[i]:
                    row_results['cost_per_ses'] = finances[i].replace('Cost per Session: ', '')
        else:
            print('Cost per Session field missing')
            row_results['cost_per_ses'] = 'missing'

        if any('Sliding Scale' in s for s in finances):
            for i in range(len(finances)):
                if 'Sliding Scale: ' in finances[i]:
                    row_results['sliding_scale'] = finances[i].replace('Sliding Scale: ', '')
        else:
            print('Sliding Scale field missing')
            row_results['sliding_scale'] = 'missing'

        for i in range(len(finances)):
            if 'Pay By' in finances[i]:
                try:
                    row_results['pay_by'] = finances[i+1]
                except: 
                    print('missing pay_by')
                    row_results['pay_by'] = 'missing'

        if 'Accepted Insurance Plans' not in finances:
                print('insurance field missing')
                row_results['accepted_insurance'] = 'missing'
        else:
            for i in range(len(finances)):
                if 'Accepted Insurance Plans' in finances[i]:
                    row_results['accepted_insurance'] = [i for i in finances[i+1:]]
       
            
        #row_results['quals'] = soup.find(class_ = 'profile-qualifications details-section top-border').text
        check = ['Years in Practice:','School:',  'Year Graduated:', 'License and State:']
        
        quals = soup.find(class_ = 'profile-qualifications details-section top-border').text
        quals_list = [item for item in (item.strip() for item in quals.split('\n') if item) if item][1:]
        if len(quals_list) > 8:
            print('check this link')
        for i in check:
            if i in quals_list:
                row_results[i] = quals_list[quals_list.index(i)+1]
            else:
                row_results[i] = 'missing'
                print(i + ' field missing')
        
        try:
            row_results['specs'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-top').text.split('\n') if item) if item])
        except:
            row_results['specs'] = 'missing'
            print('specs missing')
            
        try:
            row_results['issues'] = ', '.join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-issues').text.split('\n') if item) if item][1:])
        except:
            row_results['issues'] = 'missing'
            print('issues missing')
        try:
            row_results['mental_health'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-mental-health').text.split('\n') if item) if item][1:])
            
        except:
            print('mental health field missing')
            row_results['mental_health'] = 'missing'
        try:
            row_results['sexuality'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-sexuality').text.split('\n') if item) if item][1:])
        except:
            print('sexuality field missing')
            row_results['sexuality'] = 'missing'
        try:  
            row_results['client_focus_age'] = ", ".join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-age-focus').text.split('\n') if item) if item][1:])
        except:
            print('client focus age field missing')
            row_results['client_focus_age'] = 'missing'
        try:
            row_results['client_focus_communities'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-categories').text.split('\n') if item) if item][1:])
        except:
            print('client focus communities field missing')
            row_results['client_focus_communities'] = 'missing'
        try:
            row_results['treatment_approach_types_of_ther'] = ", ".join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-treatment-orientation').text.split('\n') if item) if item][1:])
        except:
            print('treatment approach types of therapy field missing')
            row_results['treatment_approach_types_of_ther'] = 'missing'
        try:
            row_results['treatment_approach_modality'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-modality').text.split('\n') if item) if item][1:])
        except:
            print('treatment approach modality field missing')
            row_results['treatment_approach_modality'] = 'missing'
        try:
            row_results['cert'] = re.sub('Certificate: ', "", [item for item in (item.strip() for item in soup.find(class_ = 'profile-additional-credentials details-section top-border').text.split('\n') if item) if item][1])

        except:
            print('additional credentials certificate field missing')
            row_results['cert'] = 'missing'
        try:
            row_results['cert_date'] = re.sub('Certificate Date: ', "", [item for item in (item.strip() for item in soup.find(class_ = 'profile-additional-credentials details-section top-border').text.split('\n') if item) if item][2])
        except:
            print('additional credentials certificate date field missing')
            row_results['cert_date'] = 'missing'
        try:   
            row_results['ethnicity'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-subcat attributes-ethnicity-focus').text.split('\n') if item) if item][1:])
        except:
            print('ethnicity field missing')
            row_results['ethnicity'] = 'missing'
        try:
            row_results['language'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-subcat attributes-language').text.split('\n') if item) if item][1:])
        except:
            print('language field is missing')
            row_results['language'] = 'missing'
        try:
            row_results['faith'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = 'spec-subcat attributes-religion').text.split('\n') if item) if item][1:])
        except:
            print('faith field is missing')
            row_results['faith'] = 'missing'
        try:
            row_results['tele'] = ", ".join([item for item in (item.strip() for item in soup.find(class_ = "profile-phone-online-conult icon-teletherapy-consult cursor-pointer").text.split('\n') if item) if item])
        except:
            row_results['tele'] = 'missing'
        try:
            row_results['no_tele'] = ", ".join([item for item in (item.strip() for item in soup.find(class_="profile-phone-online-conult icon-phone-online-consult cursor-pointer").text.split('\n') if item) if item])
        except:
            row_results['no_tele'] = 'missing'
        driver.quit()
        final_df = final_df.append(row_results, ignore_index=True)
        print('---'*10)       

In [5]:
scraper(all_therapists)

1
https://www.psychologytoday.com/us/therapists/alabama/484654?sid=5f02139e5041f&ref=1&tr=ResultsProfileBtn
Sliding Scale field missing
sexuality field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
2
https://www.psychologytoday.com/us/therapists/alabama/252149?sid=5f02139e5041f&ref=2&tr=ResultsProfileBtn
Sliding Scale field missing
School: field missing
Year Graduated: field missing
sexuality field missing
client focus communities field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
------------------------------
3
https://www.psychologytoday.com/us/therapists/alabama/74096?sid=5f02139e5041f&ref=3&tr=ResultsProfileBtn
ethnicity field missing
language field is missing
faith field is missing
------------------

------------------------------
24
https://www.psychologytoday.com/us/therapists/alaska/121775?sid=5f0213a7b5407&ref=4&tr=ResultsProfileBtn
Cost per Session field missing
Sliding Scale field missing
Years in Practice: field missing
client focus communities field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
25
https://www.psychologytoday.com/us/therapists/alaska/482143?sid=5f0213a7b5407&ref=5&tr=ResultsProfileBtn
insurance field missing
Years in Practice: field missing
School: field missing
Year Graduated: field missing
License and State: field missing
sexuality field missing
client focus communities field missing
language field is missing
faith field is missing
------------------------------
26
https://www.psychologytoday.com/us/therapists/alaska/50491?sid=5f0213a7b5407&ref=6&tr=ResultsProfileBtn
Years in Practice: fiel

street address field missing
Sliding Scale field missing
ethnicity field missing
language field is missing
------------------------------
46
https://www.psychologytoday.com/us/therapists/arizona/169167?sid=5f0213b16ac00&ref=6&tr=ResultsProfileBtn
Sliding Scale field missing
mental health field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
47
https://www.psychologytoday.com/us/therapists/arizona/454045?sid=5f0213b16ac00&ref=7&tr=ResultsProfileBtn
insurance field missing
Years in Practice: field missing
mental health field missing
sexuality field missing
client focus communities field missing
treatment approach types of therapy field missing
treatment approach modality field missing
ethnicity field missing
language field is missing
------------------------------
48
https://www.psychologytoday.com/us/therapists/arizona/281

------------------------------
67
https://www.psychologytoday.com/us/therapists/arkansas/707973?sid=5f0213bb3b084&ref=7&tr=ResultsProfileBtn
street address field missing
insurance field missing
sexuality field missing
faith field is missing
------------------------------
68
https://www.psychologytoday.com/us/therapists/arkansas/262194?sid=5f0213bb3b084&ref=8&tr=ResultsProfileBtn
ethnicity field missing
language field is missing
------------------------------
69
https://www.psychologytoday.com/us/therapists/arkansas/765667?sid=5f0213bb3b084&ref=9&tr=ResultsProfileBtn
Sliding Scale field missing
Years in Practice: field missing
Year Graduated: field missing
mental health field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
70
https://www.psychologytoday.com/us/therapists/arkansas/226057?sid=5f0213bb3b084&ref=10&tr=ResultsP

Years in Practice: field missing
mental health field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
91
https://www.psychologytoday.com/us/therapists/california/103502?sid=5f0213c524af6&ref=11&tr=ResultsProfileBtn
Sliding Scale field missing
insurance field missing
sexuality field missing
ethnicity field missing
language field is missing
------------------------------
92
https://www.psychologytoday.com/us/therapists/california/142961?sid=5f0213c524af6&ref=12&tr=ResultsProfileBtn
Sliding Scale field missing
insurance field missing
Years in Practice: field missing
client focus communities field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
93
https://www.psychologytoday.com/us/therapists/california/141393?sid=5f0213c524af6&ref=13&tr=ResultsProfileBtn
Sliding 

...

------------------------------
892
https://www.psychologytoday.com/us/therapists/vermont/249177?sid=5f0215aa5e2a8&ref=12&tr=ResultsProfileBtn
mental health field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
893
https://www.psychologytoday.com/us/therapists/vermont/196740?sid=5f0215aa5e2a8&ref=13&tr=ResultsProfileBtn
Years in Practice: field missing
mental health field missing
sexuality field missing
client focus communities field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
894
https://www.psychologytoday.com/us/therapists/vermont/259589?sid=5f0215aa5e2a8&ref=14&tr=ResultsProfileBtn
Years in Practice: field missing
mental health field missing
sexuality

------------------------------
914
https://www.psychologytoday.com/us/therapists/virginia/733282?sid=5f0215b615949&ref=14&tr=ResultsProfileBtn
Sliding Scale field missing
insurance field missing
check this link
Years in Practice: field missing
mental health field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
915
https://www.psychologytoday.com/us/therapists/virginia/468936?sid=5f0215b615949&ref=15&tr=ResultsProfileBtn
Cost per Session field missing
Sliding Scale field missing
check this link
License and State: field missing
sexuality field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
916
https://www.psychologytoday.com/us/therapists/virginia/383478?sid=5f0215b615949&ref=16&tr=ResultsProfileBtn
Cost per Session field missing
Sliding Scale field missing
client focus communities field missing
treatment approach types of therapy field missing
additional crede

Cost per Session field missing
Years in Practice: field missing
sexuality field missing
client focus communities field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
936
https://www.psychologytoday.com/us/therapists/washington/187690?sid=5f0215c89d06b&ref=16&tr=ResultsProfileBtn
mental health field missing
sexuality field missing
client focus communities field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
937
https://www.psychologytoday.com/us/therapists/washington/433176?sid=5f0215c89d06b&ref=17&tr=ResultsProfileBtn
Sliding Scale field missing
mental health field missing
sexuality field missing
client focus communities field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missi

Cost per Session field missing
Sliding Scale field missing
Years in Practice: field missing
mental health field missing
sexuality field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
957
https://www.psychologytoday.com/us/therapists/wisconsin/734468?sid=5f0215eaa213e&ref=1&tr=ResultsProfileBtn
check this link
client focus age field missing
client focus communities field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
958
https://www.psychologytoday.com/us/therapists/wisconsin/392314?sid=5f0215eaa213e&ref=2&tr=ResultsProfileBtn
check this link
mental health field missing
sexuality field missing
client focus communities field missing
treatment approach types of therapy field missing
treatment approach modal

Sliding Scale field missing
Years in Practice: field missing
additional credentials certificate field missing
additional credentials certificate date field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
978
https://www.psychologytoday.com/us/therapists/wyoming/39125?sid=5f0215fc9d2a5&ref=2&tr=ResultsProfileBtn
Sliding Scale field missing
Years in Practice: field missing
sexuality field missing
client focus communities field missing
ethnicity field missing
language field is missing
faith field is missing
------------------------------
979
https://www.psychologytoday.com/us/therapists/wyoming/252086?sid=5f0215fc9d2a5&ref=3&tr=ResultsProfileBtn
Cost per Session field missing
Sliding Scale field missing
insurance field missing
Years in Practice: field missing
School: field missing
Year Graduated: field missing
specs missing
issues missing
mental health field missing
sexuality field missing
client focus age field missing
clien

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
final_df

,statement,name,verified,degree,addr,city,state,zip,phone,cost_per_ses,sliding_scale,pay_by,accepted_insurance,specs,issues,mental_health,sexuality,client_focus_age,client_focus_communities,treatment_approach_types_of_ther,treatment_approach_modality,cert,cert_date,ethnicity,language,faith,tele,no_tele,License and State:,School:,Year Graduated:,Years in Practice:
0,Countryside Counseling is found in the country...,Daniel Ray Crawford,Verified by Psychology Today,"Licensed Professional Counselor, MA, LPC",699 Esslinger Drive,"Gurley,",Alabama,35748,(256) 585-9351,$80+,missing,"American Express, Cash, Discover, Mastercard, ...","[BlueCross and BlueShield, Currently accepting...","Behavioral Issues, Family Conflict, Emotional ...","ADHD, Addiction, Anger Management, Anxiety, As...","Impulse Control Disorders, Mood Disorders",missing,"Children (6 to 10), Preteens / Tweens (11 to 1...",Veterans,"Adlerian, Cognitive Behavioral (CBT), Family /...","Individuals, Family, Group",missing,missing,missing,missing,missing,missing,Ask about video and phone sessions,3014 Alabama,University of Alabama Birmingham,2008,12 Years
1,I enjoy working with adult individuals who are...,Kendra Lee Ann Laconsay,Verified by Psychology Today,"Psychologist, PsyD",30466 Sgt E I Boots Thomas Drive,"Spanish Fort,",Alabama,36527,(251) 732-2193,$120,missing,"Cash, Check, Mastercard, Visa","[Aetna, American Behavioral, Anthem, Beacon, B...","Testing and Evaluation, Anxiety, Learning Disa...","ADHD, Anger Management, Asperger's Syndrome, A...","Mood Disorders, Personality Disorders",missing,Adults,missing,"Christian Counseling, Cognitive Behavioral (CB...",Individuals,missing,missing,missing,missing,Christian,missing,Ask about video and phone sessions,AL 1055 Alabama,missing,missing,21 Years
2,"We work with a variety of clients and issues, ...","A Better Way, LLC",Verified by Psychology Today,"Licensed Professional Counselor, PhD, LPC, NCC",5510 Wares Ferry Rd,"Montgomery,",Alabama,36117,(334) 605-8626,$50 - $130,Yes,"American Express, Cash, Check, Discover, Maste...","[Aetna, American Behavioral, Anthem, Any manag...","Transgender, Anxiety, Depression","Behavioral Issues, Bipolar Disorder, Child or ...","Mood Disorders, Personality Disorders, Psychos...","Bisexual, Gay, Lesbian","Toddlers / Preschoolers (0 to 6), Children (6 ...","Bisexual Allied, Gay Allied, HIV / AIDS Allied...","Client Centered, Cognitive Behavioral (CBT), E...","Individuals, Couples, Group",American Counseling Association / 10138,2017,missing,missing,missing,Offers video and phone sessions,missing,2464 Alabama,Southern Illinois University,1995,14 Years
3,Ready for change? Would you like to get to the...,Bragwell Services,Verified by Psychology Today,"Licensed Professional Counselor, MA, LPC, LPC-SC",2930 Highway 237,"Phil Campbell,",Alabama,35581,(938) 238-3571,$80 - $120,Yes,"Cash, Check, Discover, Health Savings Account,...","[All Kids, American Behavioral, Anthem, BlueCr...","ADHD, Anxiety, Spirituality","Addiction, Alcohol Use, Alzheimer's, Anger Man...","Elderly Persons Disorders, Mood Disorders",missing,"Preteens / Tweens (11 to 13), Adolescents / Te...",Veterans,"Christian Counseling, Clinical Supervision and...","Individuals, Group",National Board of Certified Counselors / 50322,1997,African-American and Hispanic and Latino,missing,Christian,Offers video and phone sessions,missing,1820 Alabama,Regent University,1995,21 Years
4,I am a clinical psychologist. I work with chi...,France M Frederick,Verified by Psychology Today,"Psychologist, PhD",131 Fairhope Ave,"Fairhope,",Alabama,36532,(251) 829-7551,$80 - $180,missing,"Cash, Check, Mastercard, Visa",missing,"Marital and Premarital, Family Conflict, Child...","ADHD, Addiction, Anger Management, Anxiety, Ca...","Mood Disorders, Psychosis","Bisexual, Gay, Lesbian","Children (6 to 10), Preteens / Tweens (11 to 1...",missing,"Cognitive Behavioral (CBT), Family / Marital, ...","Individuals, Couples, Family",Postdoctoral Training in Child Psy

In [8]:
final_df.to_csv('1000_therapists.csv')